In [1]:
import pandas as pd
import numpy as np
import os
from collections import defaultdict
import matplotlib.pyplot as plt
from matplotlib import style

%matplotlib notebook


# data preprocessing


filenames=['DS0003','DS0006','DS0009','DS0012','DS0015','DS0017']
cohorts=np.arange(1,7).astype(str).tolist()

data = {}
data_size=0
# fpath='C:\\Users\\tolut\\Box Sync\\Documents\\PythonScripts\\dataIncubator\\ICPSR_23380\\'
fpath='ICPSR_23380\\'
for index, file in enumerate(filenames):
    fname = fpath+file+'\\23380-'+file[2:]+'-Data.tsv'
    temp_pd=pd.read_csv(fname,sep='\t',na_values=np.nan,dtype=str)
    temp_pd['cohort']=cohorts[index]
    data[cohorts[index]]=temp_pd
    data_size+=os.stat(fname).st_size

    
data_size=data_size/1e6

columnNames={}
for c in cohorts:
    columnNames[c]= data[c].columns

#unify column names
for index, c in enumerate(cohorts):
    prefix1='C'+ c
    prefix2='R'+ c
    prefix3='P'+ c
    st=len(prefix1)
    temp_columnNames=[ x.replace(c,'_')  if (x.startswith(prefix1)|x.startswith(prefix2) |x.startswith(prefix3)) \
                      else x  for x in data[c].columns]
    data[c].columns=temp_columnNames

#accounting for changes in column names
# change the PLREG column name in cohort 6
data['6'].rename(columns={'P_PLREG':'PLREG'},inplace=True)
data['2'].rename(columns={'C_DEP_YR':'C_DEP2YR'},inplace=True)
data['2'].rename(columns={'R_DEP_YR':'R_DEP2YR'},inplace=True)
data['1'].rename(columns={'C_SMK_00':'C_SMK100'},inplace=True)
data['1'].rename(columns={'R_SMK_00':'R_SMK100'},inplace=True)
data['1'].rename(columns={'C_WLK_BK':'C_WLK1BK'},inplace=True)
data['1'].rename(columns={'R_WLK_BK':'R_WLK1BK'},inplace=True)
data['3'].rename(columns={'C_CASE_ID':'CASE_ID'},inplace=True)
data['4'].rename(columns={'SFLAG':'STATUS'},inplace=True)
data['5'].rename(columns={'SFLAG':'STATUS'},inplace=True)
data['6'].rename(columns={'SFLAG':'STATUS'},inplace=True)

# combine
com_cols=set(data['1'])
for c in cohorts:
    com_cols=com_cols & set(data[c])

com_cols=list(com_cols)
    
c_data=data['1'][com_cols]
for c in cohorts[1:]:
    c_data=c_data.append(data[c][com_cols])

# deal with missing values and tranform ordinal variables
c_data.replace(r'\s+', np.nan, regex=True,inplace=True)
# data clean up and transformation (1)
d_dict=pd.ExcelFile('insurance.xlsx').parse('Sheet1') # arbitrary scale created for data columns
cat_dict=dict(zip(d_dict.feature,d_dict.Category))
longName_dict=dict(zip(d_dict.feature,d_dict['long name']))
Categories=list(set(cat_dict.values()))

#create transformation dictionary
trans_dict=defaultdict(dict)
for feature in d_dict.feature:
    temp_allowed = d_dict.loc[d_dict.feature==feature,'allowed']
    temp_trans=d_dict.loc[d_dict.feature==feature,'transformed']
    if not(pd.isnull(temp_trans.values) | pd.isnull(temp_allowed.values)):
        keys=temp_allowed.values[0].split(',')
        values=list(map(int,temp_trans.values[0].split(',')))
        trans_dict[feature]=defaultdict(lambda: np.nan,zip(keys,values))
        
t_data=c_data.copy() # transformed data
t_data.replace(to_replace=trans_dict,inplace=True)

#group features into classes
Computables=['Health PerceptionA','Chronic ConditionsA','Physical HealthA','Lifestyle ChoicesA','Emotional HealthA', \
            'Health PerceptionB','Chronic ConditionsB','Physical HealthB','Lifestyle ChoicesB','Emotional HealthB']
Compute_dict=defaultdict(list)
for feature in  t_data.columns:
    Compute_dict[cat_dict[feature]].append(feature)
    
#append new grouping features to table
for feature in Computables:
    temp_cols=Compute_dict.get(feature)
    t_data[feature]=t_data[temp_cols].mean(axis=1)
    #create list of predictor features and output targets
pfeatures=['PLREG','cohort']
targets=[] # target features

for c in c_data.columns:
    if c.startswith('C_'):
        pfeatures.append(c)
    if c.startswith('R_'):
        targets.append(c)


# remove unwanted features
notWanted=['C_WHOCMP','C_SRVDSP','C_RNDNUM','C_PCTCMP','R_SVLANG','SAMPLED', \
           'R_WHOCMP','R_SRVDSP','R_RNDNUM','R_PCTCMP','COHORT']

pfeatures=list(set(pfeatures)-set(notWanted))
targets=list(set(targets)-set(notWanted))

#remove invalid data and rows/columns with too many NaNs
c_data2=c_data.copy()
c_data2.drop(c_data2[c_data2.STATUS=='3'].index,inplace=True) #invalid

# surveys less than 80% complete
c_data2.drop(c_data2[c_data2.R_SRVDSP!='M10'].index,inplace=True) 
c_data2.drop(c_data2[c_data2.C_SRVDSP!='M10'].index,inplace=True) 

X_data=c_data2[pfeatures]
y_data=c_data2[targets]


X_data=X_data.applymap(lambda x: int(x) if (isinstance(x,str)) else np.nan)
y_data=y_data.applymap(lambda x: int(x) if (isinstance(x,str)) else np.nan)

# data clean up and transformation (2)
# from statistics import mode
#replace missing values with median

def custom_replace(x):
    md=np.nanmedian(x)
    return x.fillna(value=md)
    
X_data=X_data.apply(custom_replace, axis=0)
y_data=y_data.apply(custom_replace, axis=0)
# #convert categorical variables to continuous using one-hot encoding
X_data=pd.get_dummies(X_data,columns=pfeatures)

# exploratory data analyses
#transformation dictionaries
rdict={}
rdict['1']='Region I - Boston'
rdict['2']='Region II - New York'
rdict['3']='Region III - Philadelphia'
rdict['4']='Region IV - Atlanta'
rdict['5']='Region V - Chicago'
rdict['6']='Region VI - Dallas'
rdict['7']='Region VII - Kansas City'
rdict['8']='Region VIII - Denver'
rdict['9']='Region IX - San Francisco'
rdict['10']='Region X - Seattle'

racedict=dict(zip(['1','2','3'],['White','Black','Other']))
gendict=dict(zip(['1','2'],['Male','Female']))
agedict=dict(zip(['1','2','3'],['<65','65-74','>74']))
mrdict=dict(zip(['1','2'],['Married','Non-Married']))
educdict=dict(zip(['1','2','3'],['< high school or GED','high school or GED','> high school or GED']))

replace_dict={'PLREG':rdict,'C_RACE':racedict,'C_GENDER':gendict,'C_AGEGRP':agedict, \
        'C_MRSTAT':mrdict,'C_EDUC':educdict}

t_data.replace(to_replace=replace_dict,inplace=True)

t_data2=t_data.copy()
#remove incomplete rows
t_data2.drop(t_data2[t_data2.STATUS=='3'].index,inplace=True) #invalid
# surveys less than 80% complete
t_data2.drop(t_data2[t_data2.R_SRVDSP!='M10'].index,inplace=True) 
t_data2.drop(t_data2[t_data2.C_SRVDSP!='M10'].index,inplace=True) 

special=['cohort','PLREG','C_AGEGRP','C_RACE','C_GENDER','C_MRSTAT','C_EDUC','Health PerceptionA','Chronic ConditionsA',\
          'Physical HealthA','Lifestyle ChoicesA','Emotional HealthA', \
            'Health PerceptionB','Chronic ConditionsB','Physical HealthB','Lifestyle ChoicesB','Emotional HealthB']
sp_data=pd.DataFrame()
sp_data=sp_data.append(t_data2[special])

change_labels=['Health Perception','Physical Health','Emotional Health','Chronic Conditions']

for cl in change_labels:
    temp_cl1=cl+'A'
    temp_cl2=cl+'B'
#     sp_data[cl]=(c_data[temp_cl2]-c_data[temp_cl1])/c_data[temp_cl1]*100
    sp_data[cl]=sp_data[temp_cl2]/sp_data[temp_cl1]*100

# sp_data.groupby('PLREG').median()
map_data=sp_data.groupby('PLREG').median()
#visualization I
import warnings; warnings.simplefilter('ignore')
from bokeh.io import show,output_notebook
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LogColorMapper
)
from collections import defaultdict
from bokeh.palettes import Viridis6 as palette
from bokeh.plotting import figure
from bokeh.sampledata.us_states import data as states

#remove non mainland states
del states["HI"]
del states["AK"]

palette.extend(palette[0:4])
region_dict=defaultdict(list)
rate_dict={}
color_dict={}
region_dict['Region I - Boston']=['CT','ME','MA','NH','RI','VT']
region_dict['Region II - New York']=['NY','NJ','PR','VI']
region_dict['Region III - Philadelphia']=['DE','DC','MD','PA','VA','WV']
region_dict['Region IV - Atlanta']=['AL','FL','GA','KY','MS','NC','SC','TN']
region_dict['Region V - Chicago']=['IL','IN','MI','MN','OH','WI']
region_dict['Region VI - Dallas']=['AR','LA','NM','OK','TX']
region_dict['Region VII - Kansas City']=['IA','KS','MO','NE']
region_dict['Region VIII - Denver']=['CO','MT','ND','SD','UT','WY']
region_dict['Region IX - San Francisco']=['AZ','CA','GU','HI','NV']
region_dict['Region X - Seattle']=['AK','ID','OR','WA']

for index, key in enumerate(region_dict.keys()):
    rate_dict[key]=index
    color_dict[key]=palette[index]

state_xs = [states[code]["lons"] for code in states]
state_ys = [states[code]["lats"] for code in states]
fill_colors=[]
region_names=[]
region_rates=[]
state_names=[]
# health outcome indices
PHA=[]
EHA=[]
HP=[]
PHA_f=[]
EHA_f=[]
HP_f=[]

for x in states.keys():
    for key, value in region_dict.items():
        if x in value:
            region_names.append(key)
            region_rates.append(rate_dict[key])
            fill_colors.append(color_dict[key])
            state_names.append(states[x]['name'])
            PHA.append(np.round(map_data.loc[key,'Physical HealthA'],2))
            EHA.append(np.round(map_data.loc[key,'Emotional HealthA'],2))
            HP.append(np.round(map_data.loc[key,'Health PerceptionA'],2))
            PHA_f.append(np.round(map_data.loc[key,'Physical Health'],2))
            EHA_f.append(np.round(map_data.loc[key,'Emotional Health'],2))
            HP_f.append(np.round(map_data.loc[key,'Health Perception'],2))
             
source = ColumnDataSource(data=dict(
    x=state_xs,
    y=state_ys,
    name=region_names,
    sname=state_names,
    rate=region_rates,
    PHA_=PHA,
    EHA_=EHA,
    HP_=HP,
    PHA_f_=PHA_f,
    EHA_f_=EHA_f,
    HP_f_=HP_f
    
))

TOOLS = "pan,wheel_zoom,reset,hover,save"

p = figure(
    title="Health Outcomes survey, 1998-2014", tools=TOOLS,
    x_axis_location=None, y_axis_location=None,
    plot_width=1100,plot_height=800
)
p.grid.grid_line_color = None

p.patches('x', 'y', source=source,
          fill_color=fill_colors,
          fill_alpha=0.7, line_color='white', line_width=0.5)

hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [
    ("Region", "@name"),
    ("State", "@sname"),
    ("Physical Health", "@PHA_"),
    ("Emotional Health", "@EHA_"),
    ("Health Perception", "@HP_"),
    ("Physical Health Improvement", "@PHA_f_"),
    ("Emotional Health Improvement", "@EHA_f_"),
    ("Health Perception Improvement", "@HP_f_"),
    ("(Long, Lat)", "($x, $y)"),
]


from bokeh.resources import CDN
from bokeh.embed import file_html
html = file_html(p, CDN, "Figure 1")
Html_file= open("HealthMap.html","w")
Html_file.write(html)
Html_file.close()

C:\Users\tolut\Anaconda3\lib\site-packages\bokeh\util\deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)


GlyphRenderer(id='c2be0258-7bda-4796-a19d-ca16693d5222', ...)

192065